In [49]:
import nltk
import pandas as pd
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import seaborn as sns
import numpy as np
import nltk
nltk.download('wordnet')
import re

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/a0j01no/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/a0j01no/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [50]:
sia = SentimentIntensityAnalyzer()

In [51]:
res  = sia.polarity_scores("I am a good bad boy")

In [52]:
res

{'neg': 0.417, 'neu': 0.238, 'pos': 0.345, 'compound': -0.1531}

In [53]:
(list(res.keys()))[list(res.values()).index(np.max(list(res.values())))]

'neg'

In [54]:
np.max(list(res.values()))

0.417

In [55]:
tweets = pd.read_csv("Tweets-1.csv")

In [56]:
X = list(tweets.text.values)
Y = list(tweets.airline_sentiment.values)

In [71]:
count = 0
for message, sentiment in zip(tweets.text.values, tweets.airline_sentiment.values):
    res = sia.polarity_scores(message)
    sentim = (list(res.keys()))[list(res.values()).index(np.max(list(res.values())))]
    if sentim == sentiment[:3]:
        count = count+1
count/len(tweets.text.values)

0.21284153005464482

In [57]:
def preprocess(document, stem=False):
    document = document.lower()
    
    words = word_tokenize(document)
    
    words = [word for word in words if word not in stopwords.words("english")]
    
    if stem:
        words = [PorterStemmer().stem(word) for word in words]
    else:
        words = [WordNetLemmatizer().lemmatize(word, pos='v') for word in words]
    
    document = " ".join(words)
    
    return document

In [58]:
all_words = []
message_set = []
for s,l in zip(X,Y):
    #print(s)
    temp = s
    temp = re.sub(r'@[A-Za-z]+', '', temp)                                # removing words with @ signs
    temp = re.sub(r'[^\x00-\x7F]+', '', re.sub(r"http\S+", "", temp))     # removing emoji's
    temp = re.sub(r'[^\w\s]', '', temp)                                   # removing punctuations
    temp = re.sub(r'[0-9]+', '', temp)                                    # removing numbers
    filterd_words = [word for word in preprocess(temp).split() if len(word) > 3]
    if len(filterd_words) == 0:                             # removing the messages which have no words in it
        continue
    message_set.append((filterd_words, l))
    all_words.extend(filterd_words)

In [59]:
count = 0
for (message,sentiment) in message_set:
    res = sia.polarity_scores(" ".join(message))
    sentim = (list(res.keys()))[list(res.values()).index(np.max(list(res.values())))]
    if sentim == sentiment[:3]:
        count=count+1
print(count/len(message_set))

0.27375300997592017


In [60]:
sia.polarity_scores(" ".join(message_set[0][0]))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [61]:
message_set[0][0]

'plus youve commercials experience tacky'

### As you can see the sentiment intensity analyzer is not giving great results, we will have to build a model

In [72]:
count

3116

In [69]:
sentiment[:3]

'neu'

In [70]:
sia.polarity_scores(message)

{'neg': 0.0, 'neu': 0.952, 'pos': 0.048, 'compound': 0.0772}